## Transient Analysis of PDEs

In [43]:
%pylab
import math as mt
import matplotlib.pyplot as pl

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


### 1. Create Grid Points
### 2. Define Boundary Conditions
### 3. Perform Iteration

#### Heat Diffusion Equation - Conduction within a body (1D)

\begin{equation}
\frac{\partial T}{\partial t} = \alpha \frac{\partial ^2T}{\partial x^2}
\end{equation}

Task A: Heat Conduction in a Bar

In [72]:
def HeatFDifference(alp, a, b, T0, Ta, Tb, t0, tend, k, h):
    
    # x_grid points
    x = np.arange(a,b+h,h)
    Nx = len(x)
    
    # t_grid points
    t = np.arange(0,tend+k,k)
    Nt = len(t)
    
    # create the solution matrix
    T = np.ndarray((Nt,Nx))
    
    # set the initial conditions
    T[0,:] = T0
    
    c = alp * k / h**2
    print(c)
    
    # perform the iteration
    for p in range(1, Nt):
        T[p, 0] = Ta
        T[p, Nx-1] = Tb
                
        for i in range(1, Nx-1):
            T[p, i] = c*(T[p-1, i+1] - 2*T[p-1, i] + T[p-1, i-1]) + T[p-1, i]
    pl.plot(x,T[-1,:])

In [36]:
alp = 1.172e-5
a = 0
b = 0.5
T0 = 10
Ta = 50
Tb = 70
t0 = 0
tend = 3600
k = 1
h = 0.01
HeatFDifference(alp, a, b, T0, Ta, Tb, t0, tend, k, h)

Task B: Heat Conduction in a bar with a heat source

In [68]:
def HDEFDSource(alp, K, h, a, b, T0, t0, tend, dt, dx):
    
    # x_grid points
    x = np.arange(a,b+dx,dx)
    Nx = len(x)
    
    # t_grid points
    t = np.arange(0,tend+dt,dt)
    Nt = len(t)
    
    # create the solution matrix
    T = np.ndarray((Nt,Nx))
    
    # set the initial conditions
    T[0,:] = T0
    T[0, mt.ceil(Nx/2)] = 100
    
    c = alp * dt / dx**2
    print(c)
    
    # perform the iteration
    for p in range(1, Nt):
        T[p,0] = (h*Tw+K/dx*T[p-1,1]) / (h+K/dx)
        T[p,Nx-1] = (h*Tw+K/dx*T[p-1,Nx-2]) / (h+K/dx)
                
        for i in range(1, Nx-1):
            T[p, i] = c*(T[p-1, i+1] - 2*T[p-1, i] + T[p-1, i-1]) + T[p-1, i]
            T[p,mt.ceil(Nx/2)] = 100
    pl.plot(x,T[-1,:]) # plot T(x, tend)
    return c

In [69]:
alp = 1.172e-5
K = 40
h = 500
a = 0
b = 0.5
T0 = 10
Tw = 5
t0 = 0
tend = 1200
dt = 1
dx = 0.01
HDEFDSource(alp, K, h, a, b, T0, t0, tend, dt, dx)

0.11719999999999998


0.11719999999999998

Task C: Stability of the finite difference numerical method

In [73]:
alp = 1.172e-5
a = 0
b = 0.5
T0 = 10
Ta = 50
Tb = 50
t0 = 0
tend = 3600
HeatFDifference(alp, a, b, T0, Ta, Tb, t0, tend, 1, 0.01)
HeatFDifference(alp, a, b, T0, Ta, Tb, t0, tend, 1, 0.005)
HeatFDifference(alp, a, b, T0, Ta, Tb, t0, tend, 1, 0.001)

0.11719999999999998
0.46879999999999994
11.72


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in double_scalars
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


Task D: Baking a Potato in the Oven

In [78]:
def TwoDimensionalPDE(xa, xb, ya, yb, pw, dx, dy, dt, tend, Toven, Tpot, Tw, alphaAir, alphaPot):
    # position of the potato
    xap = (xb-xa)/2 - pw/2
    xbp = (xb-xa)/2 + pw/2
    yap = (yb-ya)/2 - pw/2 
    ybp = (yb-ya)/2 + pw/2
    
    # x_grid points
    x = np.arange(xa,xb+dx,dx)
    Nx = len(x)
    
    # y_grid points
    y = np.arange(ya,yb+dy,dy)
    Ny = len(y)
    
    # t_grid points
    t = np.arange(0,tend+dt,dt)
    Nt = len(t)
    
    # create the solution matrix
    T = np.ndarray((Nt,Nx,Ny))
    
    # diffusivity value matrix (allows for different alpha values throughout the grid)
    alpha = np.ndarray((Nx,Ny))
    alpha[:,:] = alphaAir
    
    # set the inital value everywhere in the oven
    T[0,:,:] = Toven
    
    # initialise the temperature of the potato
    for i in range(0,Nx):
        for j in range(0,Ny):
            if (xap<=x[i]<=xbp) and (yap<=y[j]<=ybp):
                T[0,i,j] = Tpot
                alpha[i,j] = alphapot
    # check the Courant condition
    Courant = np.max(alpha) * dt/ dx**2
    print(Courant)
    
    # compute the constant coefficient
    cx = alpha * dt / dx**2
    cy = alpha * dt / dy**2
    
    # compute the solution incrementally at subsequent time steps
    for p in range(1,Nt):
        
        # boundary conditions
        T[p,0,:] = Tw # N
        T[p,Nx-1,:] = Tw # S
        T[p,:,0] = Tw # W
        T[p,:,Ny-1] = Tw # E
        
        # do the interior nodes
        for i in range(1,Nx-1):
            for j in range(1,Ny-1):
            
            # apply the discretised equation
                T[p,i,j] = cx[i,j]*(T[p-1,i+1,j]+T[p-1,i-1,j]) + \
                           cy[i,j]*(T[p-1,i,j+1]+T[p-1,i,j-1]) + \
                           (1 - 2*cx[i,j] - 2*cy[i,j]) * T[p-1,i,j]
        
    print( np.min(T[-1,:,:]) )
        # plot surface or contour plot
    (Yg, Xg) = np.meshgrid(y,x)
    pl.contourf(Xg,Yg,T[-1,:,:])
    pl.colorbar()

In [ ]:
xa = 0
xb = 0.4
ya = 0
yb = 0.3
pw = 0.06 # potato width

dx = 0.01 # spatial increment
dy = dx
# temporal domain
dt = 1 # temporal increment
tend = 9000  # temporal span

Toven = 10 # initial Temperature of oven
Tpot = -15 # initial Temperature of potato
Tw = 180 # T of oven walls

alphaAir = 1.9e-5 # thermal diffusivity of air
alphaPot = 1.3e-7 # thermal diffusivity of potato

TwoDimensionalPDE(xa, xb, ya, yb, pw, dx, dy, dt, tend, Toven, Tpot, Tw, alphaAir, alphaPot)

0.19
